In [1]:
import pandas as pd 
import numpy as np 
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from datasets import Dataset
import warnings 
import pandas as pd
import torch
warnings.filterwarnings("ignore")

In [2]:
data=pd.read_csv('/kaggle/input/crop-dataset/Crop_recommendation.csv')
data.head()

,N,P,K,temperature,humidity,ph,rainfall,label
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from datasets import Dataset
from sklearn.preprocessing import LabelEncoder
import torch

df = pd.read_csv('/kaggle/input/crop-dataset/Crop_recommendation.csv') 

def row_to_text(row):
    return (f"The nitrogen content is {row['N']}, phosphorus is {row['P']}, "
            f"and potassium is {row['K']}, with a temperature of {row['temperature']}°C, "
            f"humidity of {row['humidity']}%, pH of {row['ph']}, "
            f"and rainfall of {row['rainfall']}mm.")

In [4]:
df['text'] = df.apply(row_to_text, axis=1)

label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

train_texts, valid_texts, train_labels, valid_labels = train_test_split(df['text'], df['label'], test_size=0.2)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_function(texts):
    return tokenizer(texts, truncation=True, padding=True)

train_encodings = tokenize_function(train_texts.tolist())
valid_encodings = tokenize_function(valid_texts.tolist())

train_dataset = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': train_labels
})

valid_dataset = Dataset.from_dict({
    'input_ids': valid_encodings['input_ids'],
    'attention_mask': valid_encodings['attention_mask'],
    'labels': valid_labels
})

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [5]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(df['label'].unique()))

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    
    return {'accuracy':acc,
            'f1' :f1   
    }

In [7]:
from transformers import TrainerCallback

class CustomCallback(TrainerCallback):
    def on_epoch_begin(self, args, state, control, **kwargs):
        print(f"\nEpoch {state.epoch + 1}/{state.num_train_epochs} started.")

## Training 

In [8]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    evaluation_strategy="steps",
    logging_dir='./logs',
    logging_steps=200,
    save_steps=200,
    load_best_model_at_end=True,
    eval_steps=200,
    warmup_steps=100,
)

data_collator = DataCollatorWithPadding(tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [9]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss,Accuracy,F1
200,2.819600,1.774108,0.384091,0.310445
400,1.270000,0.834167,0.650000,0.581602
600,0.519400,0.286404,0.877273,0.862557
800,0.177300,0.110856,0.979545,0.979306
1000,0.060400,0.096488,0.975000,0.975036


TrainOutput(global_step=1100, training_loss=0.8839680777896535, metrics={'train_runtime': 311.2828, 'train_samples_per_second': 56.54, 'train_steps_per_second': 3.534, 'total_flos': 687501065856000.0, 'train_loss': 0.8839680777896535, 'epoch': 10.0})

In [10]:
trainer.evaluate()

{'eval_loss': 0.09648821502923965,
 'eval_accuracy': 0.975,
 'eval_f1': 0.975036342825314,
 'eval_runtime': 1.5648,
 'eval_samples_per_second': 281.193,
 'eval_steps_per_second': 8.947,
 'epoch': 10.0}

In [11]:
model.save_pretrained('./fine_tuned_bert_model')
tokenizer.save_pretrained('./fine_tuned_bert_model')

('./fine_tuned_bert_model/tokenizer_config.json',
 './fine_tuned_bert_model/special_tokens_map.json',
 './fine_tuned_bert_model/vocab.txt',
 './fine_tuned_bert_model/added_tokens.json')

## Predicting Crop from features 

In [12]:
def predict(text):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(device)
    model.to(device)
    
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=-1).item()
    
    return label_encoder.inverse_transform([predicted_class])[0]

In [13]:
predicted_crop = predict("The nitrogen content is 90, phosphorus is 42, and potassium is 43, pH is 6.50 with a temperature of 20.86896°C.")
print(predicted_crop)

jute


## Description of the predicted crop 

In [14]:
# # Load model directly
# from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer_phi = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct", trust_remote_code=True)
# model_phi = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3.5-mini-instruct", trust_remote_code=True)

tokenizer_config.json:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

In [16]:
!pip install bitsandbytes --no-cache-dir

In [17]:
import bitsandbytes as bnb
print(bnb.__version__)  # Should match the latest version

0.44.1


In [18]:
pip install -U transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 63.6 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.9/330.9 kB 28.1 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.34.2
    Uninstalling accelerate-0.34.2:
      Successfully uninstalled accelerate-0.34.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.45.1
    Uninstalling transformers-4.45.1:
      Successfully uninstalled transformers-4.45.1
Note: you may need to restart the kernel to use updated packages.


In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig

config = AutoConfig.from_pretrained("microsoft/Phi-3.5-mini-instruct")

# Load specific layers
model = AutoModelForCausalLM.from_config(config)


# # Move the model to CPU
# model.to('cpu')

# # Apply dynamic quantization
# model = torch.quantization.quantize_dynamic(
#     model,  # the model to quantize
#     {torch.nn.Linear},  # quantize only the Linear layers
#     dtype=torch.qint8  # quantize to 8-bit integers
# )

In [2]:
model.save_pretrained('./fine_tuned_phi_model')
tokenizer.save_pretrained('./fine_tuned_phi_model')

('./fine_tuned_phi_model/tokenizer_config.json',
 './fine_tuned_phi_model/special_tokens_map.json',
 './fine_tuned_phi_model/tokenizer.model',
 './fine_tuned_phi_model/added_tokens.json',
 './fine_tuned_phi_model/tokenizer.json')

In [15]:
input_text = f"The predicted crop is {predicted_crop}. Write a paragraph about its unique characteristics, uses, and its importance in agriculture."

input_ids = tokenizer_phi.encode(input_text, return_tensors="pt")
outputs = model_phi.generate(input_ids=input_ids, max_length=200, num_beams=4, early_stopping=True)

generated_text = tokenizer_phi.decode(outputs[0], skip_special_tokens=True)
print(generated_text)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.


The predicted crop is jute. Write a paragraph about its unique characteristics, uses, and its importance in agriculture.

# Answer
Jute, often referred to as the 'golden fiber,' is a long, soft, shiny bast fiber that can be spun into coarse, strong threads. It is one of the most affordable natural fibers and is second only to cotton in the amount produced and the variety of products it can be used for. Jute fibers are composed primarily of the plant materials cellulose and lignin.

Jute has several unique characteristics that make it highly valued in agriculture and industry. It is biodegradable, renewable, and has a low production cost, which makes it an eco-friendly alternative to synthetic fibers. The fiber is known for its high tensile strength, durability, and absorbency, which makes it suitable for a wide


In [17]:
import os
import shutil

# Define the path to the working directory
working_dir = '/kaggle/working/'

# Remove all files and directories in the working directory
for item in os.listdir(working_dir):
    item_path = os.path.join(working_dir, item)
    if os.path.isdir(item_path):
        shutil.rmtree(item_path)  # Remove directory
    else:
        os.remove(item_path)  # Remove file

print("All files and directories in the working directory have been cleared.")


All files and directories in the working directory have been cleared.


In [18]:
model_phi.save_pretrained('./fine_tuned_phi_model')
tokenizer_phi.save_pretrained('./fine_tuned_phi_model')

('./fine_tuned_phi_model/tokenizer_config.json',
 './fine_tuned_phi_model/special_tokens_map.json',
 './fine_tuned_phi_model/tokenizer.model',
 './fine_tuned_phi_model/added_tokens.json',
 './fine_tuned_phi_model/tokenizer.json')